## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import matplotlib.pyplot as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cabo San Lucas,MX,22.8909,-109.9124,72.61,63,0,1.99,clear sky
1,1,Atuona,PF,-9.8000,-139.0333,78.33,79,11,21.99,light rain
2,2,Krasnoselkup,RU,65.7000,82.4667,-15.05,100,96,4.07,overcast clouds
3,3,Busselton,AU,-33.6500,115.3333,68.04,71,16,7.43,few clouds
4,4,Narsaq,GL,60.9167,-46.0500,-0.18,70,63,5.70,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 14
What is the maximum temperature you would like for your trip? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Busselton,AU,-33.6500,115.3333,68.04,71,16,7.43,few clouds
7,7,Ushuaia,AR,-54.8000,-68.3000,49.37,89,100,2.13,overcast clouds
15,15,Mimarsinan,TR,41.0032,28.5377,45.99,84,12,8.34,few clouds
20,20,Mataura,NZ,-46.1927,168.8643,57.47,53,97,3.80,overcast clouds
21,21,Port Elizabeth,ZA,-33.9180,25.5701,69.10,84,46,18.70,scattered clouds
22,22,Yanliang,CN,34.6592,109.2292,29.16,86,100,7.31,light snow
23,23,Tongren,CN,27.7172,109.1853,39.47,87,100,4.97,overcast clouds
25,25,Saldanha,ZA,-33.0117,17.9442,68.07,70,43,4.34,scattered clouds
28,28,Ostrovnoy,RU,68.0531,39.5131,15.49,84,100,28.79,overcast clouds
30,30,Mpanda,TZ,-6.3655,31.0409,63.41,97,91,1.48,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False
15,False,False,False,False,False,False,False,False,False,False
20,False,False,False,False,False,False,False,False,False,False
21,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
684,False,False,False,False,False,False,False,False,False,False
685,False,False,False,False,False,False,False,False,False,False
686,False,False,False,False,False,False,False,False,False,False
687,False,False,False,False,False,False,False,False,False,False


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Busselton,AU,68.04,few clouds,-33.6500,115.3333,
7,Ushuaia,AR,49.37,overcast clouds,-54.8000,-68.3000,
15,Mimarsinan,TR,45.99,few clouds,41.0032,28.5377,
20,Mataura,NZ,57.47,overcast clouds,-46.1927,168.8643,
21,Port Elizabeth,ZA,69.10,scattered clouds,-33.9180,25.5701,
22,Yanliang,CN,29.16,light snow,34.6592,109.2292,
23,Tongren,CN,39.47,overcast clouds,27.7172,109.1853,
25,Saldanha,ZA,68.07,scattered clouds,-33.0117,17.9442,
28,Ostrovnoy,RU,15.49,overcast clouds,68.0531,39.5131,
30,Mpanda,TZ,63.41,overcast clouds,-6.3655,31.0409,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Busselton,AU,68.04,few clouds,-33.6500,115.3333,Observatory Guest House
7,Ushuaia,AR,49.37,overcast clouds,-54.8000,-68.3000,Albatross Hotel
15,Mimarsinan,TR,45.99,few clouds,41.0032,28.5377,Otel Pelikan
20,Mataura,NZ,57.47,overcast clouds,-46.1927,168.8643,Ellie's Villa
21,Port Elizabeth,ZA,69.10,scattered clouds,-33.9180,25.5701,39 On Nile Guest House
...,...,...,...,...,...,...,...
684,Lichinga,MZ,61.09,moderate rain,-13.3128,35.2406,Residencial Bendiak
685,Nishihara,JP,58.68,few clouds,26.1842,127.7558,Minshuku Agaihama
686,Skjervoy,NO,30.24,light snow,70.0311,20.9714,Skjervøy Byggeservice AS
687,Xingyi,CN,46.58,overcast clouds,24.1877,102.8170,


In [10]:
hotel_df.loc[(hotel_df["Hotel Name"] == ""), "Hotel Name"]=np.nan

hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Busselton,AU,68.04,few clouds,-33.6500,115.3333,Observatory Guest House
7,Ushuaia,AR,49.37,overcast clouds,-54.8000,-68.3000,Albatross Hotel
15,Mimarsinan,TR,45.99,few clouds,41.0032,28.5377,Otel Pelikan
20,Mataura,NZ,57.47,overcast clouds,-46.1927,168.8643,Ellie's Villa
21,Port Elizabeth,ZA,69.10,scattered clouds,-33.9180,25.5701,39 On Nile Guest House
...,...,...,...,...,...,...,...
684,Lichinga,MZ,61.09,moderate rain,-13.3128,35.2406,Residencial Bendiak
685,Nishihara,JP,58.68,few clouds,26.1842,127.7558,Minshuku Agaihama
686,Skjervoy,NO,30.24,light snow,70.0311,20.9714,Skjervøy Byggeservice AS
687,Xingyi,CN,46.58,overcast clouds,24.1877,102.8170,NaN


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("WeatherPy_vacation_map.png")

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>